In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Activation, Dense
from numpy import average
from numpy import array
from keras.models import clone_model
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from sklearn.metrics import confusion_matrix
import sklearn

EPOCHS = 5
NWORKERS = 5
EPOCHS_WORKER = 50

def fedAvg(members):
  n_layers = len(members[0].get_weights())
  weights = [1.0/len(members) for i in range(1, len(members)+1)]
  avg_model_weights = list()
  for layer in range(n_layers):
    layer_weights = array([model.get_weights()[layer] for model in members])
    avg_layer_weights = average(layer_weights, axis=0, weights=weights)
    avg_model_weights.append(avg_layer_weights)
  model = clone_model(members[0])
  model.set_weights(avg_model_weights)
  model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return model

def geraNModelosCopiaFederado(modeloFederado, n):
  modelos = [clone_model(modelFederado) for i in range(n)]
  for modelo in modelos:
    modelo.set_weights(modelFederado.get_weights())
    modelo.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return modelos

caminho_dataset = 'dataset/'
dataset = ['data01.csv','data02.csv']

for data in dataset:
  df = pd.read_csv(caminho_dataset + data)
  df = df.sample(frac=1).reset_index(drop=True)
  x = df.drop('class', axis=1)
  inputs = np.asarray(x)
  inputs = MinMaxScaler().fit_transform(inputs)
  labels = np.asarray(df['class'])
  labels = to_categorical(labels)

  lista_dataset_x = []
  lista_dataset_y = []
  lista_workers = []
  lista_dataset_federado = []

  for i in range(NWORKERS):
    inicio = i * int(len(inputs) / NWORKERS)
    fim = (i+1) * int(len(inputs) / NWORKERS)

    x = inputs[inicio:fim]
    y = labels[inicio:fim]
    lista_dataset_x.append(x)
    lista_dataset_y.append(y)

  modelFederado = Sequential()
  modelFederado.add(Dense(500, activation='relu', input_dim=21))
  modelFederado.add(Dense(100, activation='relu'))
  modelFederado.add(Dense(50, activation='relu'))
  modelFederado.add(Dense(2, activation='softmax'))

  modelos = geraNModelosCopiaFederado(modelFederado, NWORKERS)
  for i in range(1, EPOCHS + 1):
    print('Epoca [{:2d}/{:2d}]'.format(i, EPOCHS))
    for j in range(len(modelos)):
      history = modelos[j].fit(lista_dataset_x[j], lista_dataset_y[j], epochs=EPOCHS_WORKER, verbose=0)
      print('\tTreinou modelo: [{:2d}/{:2d}] da epoca [{:2d}] (acc {:.6f} \tLoss: {:.6f})'.format(j, len(modelos), i, history.history['accuracy'][0], history.history['loss'][0]))
    modelFederado = fedAvg(modelos)
    modelos = geraNModelosCopiaFederado(modelFederado, NWORKERS)
    
  modelFederado.save('saveModel/model' + str(data))


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
import graphviz
from IPython import embed
import pickle
import os
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import precision_score, accuracy_score
import joblib
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier

saida = 'saida_predictions/'
caminho = 'predictions/'

dataset = ['data01.csv','data02.csv']

for data in dataset:
    arquivoPredictions = pd.read_csv(caminho + data)
    lenArqPred = len(arquivoPredictions)
    predNorm = []
    predAtk = []
    predClasse = []
    for predNovo in range(lenArqPred):
        predNorm.append(float(arquivoPredictions['prob_norm'][predNovo]))  # NORMAL
        predAtk.append(float(arquivoPredictions['prob_atk'][predNovo]))  # ATAQUE
        predClasse.append(int(arquivoPredictions['classe'][predNovo]))  # CLASSE

    arq_write = open(saida + 'saida_' + data + '.csv', 'w')
    arq_write.write('limiarNorm;limiarAtk;TN;FP;FN;TP;REJ;TNR;TPR;AVG;REJ_N;ERRO;\n')

    limiarNorm = 0.0
    for limiarNormQtd in range(100): # ZERA LIMIAR PARA TESTAR
        limiarAtk = 0.0
        for limiarAtkQtd in range(100):

            TN = 0
            FP = 0
            TP = 0
            FN = 0
            REJ = 0

            for pred in range(lenArqPred):
                ### PROB 1 == ATK
                ### PROB 0 == NORM

                if (predAtk[pred] >= predNorm[pred]):
                    if (predAtk[pred] >= limiarAtk):
                        if (predClasse[pred] == 1):
                            TP += 1
                        else:
                            FP += 1
                    else:
                        REJ += 1
                else:
                    if (predNorm[pred] >= limiarNorm):
                        if (predClasse[pred] == 0):
                            TN += 1
                        else:
                            FN += 1
                    else:
                        REJ += 1

            try:
                TNR = TN / (TN + FP)
            except:
                TNR = 0
            try:
                TPR = TP / (TP + FN)
            except:
                TPR = 0
            try:
                AVG = (TNR + TPR) / 2
            except:
                AVG = 0
            try:
                REJ_N = REJ / (TN + FP + FN + TP + REJ)
            except:
                REJ_N = 0
            try:
                ERRO = ((FP / (FP + TN)) + (FN / (FN + TP))) / 2
            except:
                ERRO = 0

            print('Lim_N:' + str(limiarNorm) + ' Lim_A:' + str(limiarAtk) + ' TN:' + str(TN) + ' FP:' + str(
                FP) + ' FN:' + str(FN) + ' TP:' + str(TP) + ' REJ:' + str(REJ))
            print('ERRO: ' + str(ERRO) + ' | TNR ' + str(TNR) + ' | TPR ' + str(TPR))

            embed()

            arq_write.write(
                str(limiarNorm) + ';' + str(limiarAtk) + ';' +
                str(TN) + ';' + str(FP) + ';' + str(FN) + ';' + str(TP) + ';' + str(REJ) + ';' +
                str(TNR) + ';' + str(TPR) + ';' + str(AVG) + ';' + str(REJ_N) + ';' + str(ERRO) + ';\n')

            limiarAtk += 0.01
        limiarNorm += 0.01
    arq_write.close()

In [ ]:
from IPython import embed

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Activation, Dense
from numpy import average
from numpy import array
from keras.models import clone_model
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from sklearn.metrics import confusion_matrix


EPOCHS = 5
NWORKERS = 5
EPOCHS_WORKER = 50

def geraNModelosCopiaFederado(modeloFederado, n):
  modelos = [clone_model(modelFederado) for i in range(n)]
  for modelo in modelos:
    modelo.set_weights(modelFederado.get_weights())
    modelo.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return modelos

def fedAvg(members):
  n_layers = len(members[0].get_weights())
  weights = [1.0/len(members) for i in range(1, len(members)+1)]
  avg_model_weights = list()
  for layer in range(n_layers):
    layer_weights = array([model.get_weights()[layer] for model in members])
    avg_layer_weights = average(layer_weights, axis=0, weights=weights)
    avg_model_weights.append(avg_layer_weights)
  model = clone_model(members[0])
  model.set_weights(avg_model_weights)
  model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return model


caminho_dataset = 'dataset/'
dataset = ['data01.csv']
modelAntigo = '10'
modelNovo = '11'

for data in dataset:
  df = pd.read_csv(caminho_dataset + data)
  df = df.sample(frac=1).reset_index(drop=True)
  x = df.drop('class', axis=1)
  x = x.drop('MAWILAB_taxonomy', axis=1)
  x = x.drop('MAWILAB_label', axis=1)
  x = x.drop('MAWILAB_nbDetectors', axis=1)
  x = x.drop('MAWILAB_distance', axis=1)
  x = x.drop('ORUNADA_numberOfDifferentDestinations', axis=1)
  x = x.drop('ORUNADA_numberOfDifferentServices', axis=1)
  inputs = np.asarray(x)

  inputs = MinMaxScaler().fit_transform(inputs)
  labels = np.asarray(df['class'])
  labels = to_categorical(labels)

  lista_dataset_x = []
  lista_dataset_y = []
  lista_workers = []
  lista_dataset_federado = []

  for i in range(NWORKERS):
    inicio = i * int(len(inputs) / NWORKERS)
    fim = (i+1) * int(len(inputs) / NWORKERS)

    x = inputs[inicio:fim]
    y = labels[inicio:fim]
    lista_dataset_x.append(x)
    lista_dataset_y.append(y)

  modelFederado = keras.models.load_model('saveModel/model' + modelAntigo)
  modelos = geraNModelosCopiaFederado(modelFederado, NWORKERS)

  for i in range(1, EPOCHS + 1):
    print('Epoca [{:2d}/{:2d}]'.format(i, EPOCHS))
    for j in range(1):
      history = modelos[j].fit(lista_dataset_x[j], lista_dataset_y[j], epochs=EPOCHS_WORKER, verbose=0)
      print('\tTreinou modelo: [{:2d}/{:2d}] da epoca [{:2d}] (acc {:.6f} \tLoss: {:.6f})'.format(j, len(modelos), i, history.history['accuracy'][0], history.history['loss'][0]))
    modelFederado = fedAvg(modelos)
    modelos = geraNModelosCopiaFederado(modelFederado, NWORKERS)

  ### SAVE MODEL ###
  modelFederado.save('saveModel/model' + modelNovo)
